<a href="https://colab.research.google.com/github/GokayToga/FloodNet_Challange-GT/blob/main/FloodNet_Challange_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do:

*   resize the images to 713x713 ✅
*   Augment the images with random shuffling, scaling, flipping, and random rotation ✅
*  First Build basic U-net to experiment
*  Then PSPNet, ENet, and DeepLabv3+ can be implemented and experimented on.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Libraries**

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import cv2
import imageio

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from collections import Counter

from tensorflow.image import resize as tf_resize
from tensorflow import io as tf_io
from tensorflow import image as tf_image

from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
!pip3 install tqdm

## Data Resizing and Augmentation

In [ ]:
Data_struct = """
|
│   class_mapping.csv
│
└──>Test
│   │
│   └──>Labeled img (.png)
│       │
│       └──> images
│   │
│   └──>Unlabeled img (.jpg)
│       │
│       └──> images
└──>Train
│   │
│   └──>Labeled img (.png)
│       │
│       └──> images
│   │
│   └──>Unlabeled img (.jpg)
│       │
│       └──> images
└──>val
    │
    └──>Labeled img (.png)
        │
        └──> images
    │
    └──>Unlabeled img (.jpg)
        │
        └──> images
"""

In [ ]:
RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize(path, resize=RESIZE):
  #used_path = os.path.join(drive_path, path) #path for diffent datasets
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()

  Aug_parameters = ImageDataGenerator(   #augmentation parameters
      rescale=1./255,          # Scaling
      horizontal_flip=True,    # Random horizontal flip
      vertical_flip=True,      # Random vertical flip
      rotation_range=45        # Random rotation in the range [-45, 45] degrees
  )

  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))
    #print( length )

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + img_name
      # Read and resize the image
      img = cv2.imread(img_path)

      # Check if the image was read successfully
      if img is not None:
        img = cv2.resize(img, resize)

        # Reshape to meet the requirements of ImageDataGenerator
        img = img.reshape((1,) + img.shape)

        # Generate augmented images
        for batch in Aug_parameters.flow(img, batch_size=1, save_to_dir=local_path, save_prefix=img_name.split('.')[0], save_format=save_format):
          break  # Stop after one augmented image (to avoid infinite loop)
      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


# Data resizer :
- (Because of the problems in augmentation we countinue with only resizing)

In [ ]:


RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize2(path, resize=RESIZE):
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()


  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + "/" + img_name
      save_path = local_path + path + "/" + img_name

      try:
          img = imageio.imread(img_path)
      except Exception as e:
          print(f"Error: {e}")
          continue

      # Check if the image was read successfully
      if img is not None:

        img = cv2.resize(img, resize)
        imageio.imwrite(save_path, img)

      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


In [ ]:
#Test
Augment_Resize2("/test/test-label-img")
Augment_Resize2("/test/test-org-img")

#Train
Augment_Resize2("/train/train-label-img")
Augment_Resize2("/train/train-org-img")

#Val
Augment_Resize2("/val/val-label-img")
Augment_Resize2("/val/val-org-img")



  0%|          | 0/448 [00:00<?, ?it/s]<ipython-input-3-d434104c1896>:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)
100%|██████████| 450/450 [02:45<00:00,  2.73it/s]


-Moving the data to Drive

In [ ]:
!mv /content/Augmented_Data /content/drive/MyDrive


-Downloading the data

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/Augmented_Data')

## Transforming and Augmenting Data

In [ ]:
SEGMENTS = {'Background':0,'Building-flooded':1,'Building-non-flooded':2,'Road-flooded':3,'Road-non-flooded':4,
         'Water':5,'Tree':6,'Vehicle':7,'Pool':8,'Grass':9}
DIMENSIONS = (713,713)

In [ ]:

Aug_parameters = ImageDataGenerator(   #augmentation parameters
      rescale=1./255,          # Scaling
      horizontal_flip=True,    # Random horizontal flip
      vertical_flip=True,      # Random vertical flip
      rotation_range=45        # Random rotation in the range [-45, 45] degrees
  )

In [ ]:

class SegDataset(tf.keras.utils.Sequence):
    def __init__(self, x_paths, y_labels, batch_size, transform=None):
        self.x_paths = x_paths
        self.y_labels = y_labels
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return len(self.x_paths) // self.batch_size

    def __getitem__(self, index):
        batch_x_paths = self.x_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_y_labels = self.y_labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [cv2.imread(path) for path in batch_x_paths]
        batch_labels = batch_y_labels

        if self.transform:
            batch_images = self.transform.flow(batch_images, batch_size=len(batch_images), shuffle=False)

        batch_images = [cv2.resize(img, DIMENSIONS) / 255.0 for img in batch_images]

        return tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_labels, dtype=tf.int64)

class SegDataset3(tf.keras.utils.Sequence):#USED
    def __init__(self, x_paths, y_labels, batch_size, transform=None):
        self.x_paths = x_paths
        self.y_labels = y_labels
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return len(self.x_paths) // self.batch_size

    def __getitem__(self, index):
        batch_x_paths = self.x_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_y_labels = self.y_labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [cv2.imread(path) for path in batch_x_paths]
        batch_masks = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in batch_y_labels]

        if self.transform:
            # Create an instance of ImageDataGenerator and apply the flow method
            datagen = self.transform.flow(np.array(batch_images), batch_size=len(batch_images), shuffle=False)
            augmented_images = datagen.next()

            augmented_masks = np.array(batch_masks)

            batch_images = augmented_images
            batch_masks = augmented_masks

        #batch_images = [cv2.resize(img, DIMENSIONS) / 255.0 for img in batch_images]
        batch_images = [cv2.resize(img.astype(np.uint8), DIMENSIONS) / 255.0 for img in batch_images]
        #batch_masks = [cv2.resize(mask, DIMENSIONS, interpolation=cv2.INTER_NEAREST) for mask in batch_masks]
        batch_masks = [cv2.resize(mask.astype(np.uint8), DIMENSIONS, interpolation=cv2.INTER_NEAREST) for mask in batch_masks]


        return tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_masks, dtype=tf.int64)


class SegDataset4(tf.keras.utils.Sequence):#for resizing if needed for training performance
    def __init__(self, x_paths, y_labels, batch_size, transform=None, target_size=(256, 256)):
        self.x_paths = x_paths
        self.y_labels = y_labels
        self.batch_size = batch_size
        self.transform = transform
        self.target_size = target_size  # Set  desired target size

    def __len__(self):
        return len(self.x_paths) // self.batch_size

    def __getitem__(self, index):
        batch_x_paths = self.x_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_y_labels = self.y_labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [cv2.imread(path) for path in batch_x_paths]
        batch_masks = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in batch_y_labels]

        # Resize images and masks
        batch_images = [cv2.resize(img, self.target_size) / 255.0 for img in batch_images]
        batch_masks = [cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST) for mask in batch_masks]

        if self.transform:
            # Create an instance of ImageDataGenerator and apply the flow method
            datagen = self.transform.flow(np.array(batch_images), batch_size=len(batch_images), shuffle=False)
            augmented_images = datagen.next()

            augmented_masks = np.array(batch_masks)

            batch_images = augmented_images
            batch_masks = augmented_masks

        return tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_masks, dtype=tf.int64)


#one hot encoding for later use
class SegDataset2(tf.keras.utils.Sequence):
    def __init__(self, x_paths, y_labels, batch_size, num_classes = 10, transform=None):
        self.x_paths = x_paths
        self.y_labels = y_labels
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.transform = transform

    def __len__(self):
        return len(self.x_paths) // self.batch_size

    def __getitem__(self, index):
        batch_x_paths = self.x_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_y_labels = self.y_labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [cv2.imread(path) for path in batch_x_paths]
        batch_labels = np.array(batch_y_labels)  # Convert to numpy array for easier manipulation

        if self.transform:
            batch_images = self.transform.flow(batch_images, batch_size=len(batch_images), shuffle=False)

        batch_images = [cv2.resize(img, DIMENSIONS) / 255.0 for img in batch_images]

        # One-hot encode the labels
        batch_labels_one_hot = tf.one_hot(batch_labels, self.num_classes)

        return tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_labels_one_hot, dtype=tf.float32)

In [ ]:
#Data pathways or sm
#path : /content/drive/MyDrive/Augmented_Data
#specific path : /content/drive/MyDrive/Augmented_Data/test/test-label-img

##Model

In [ ]:
!pip3 install --upgrade tensorflow



*   First Configuration



In [ ]:

def create_paths(base_path, dataset_type):
    labeled_path = base_path + '/' + dataset_type

    image_paths = []
    label_paths = []

    for class_folder in os.listdir(labeled_path):
        if not class_folder.startswith('.'):  # Exclude hidden folders
            class_path = labeled_path

            if os.path.isdir(class_path) and class_folder == (dataset_type + '-org-img'):
                org_images_path = class_path + '/' + class_folder

                if os.path.isdir(org_images_path):
                    image_paths += [org_images_path + '/' + filename for filename in os.listdir(org_images_path)]

            if os.path.isdir(class_path) and class_folder == (dataset_type + '-label-img'):
                labeled_images_path = class_path + '/' + class_folder

                if os.path.isdir(labeled_images_path):
                    label_paths += [labeled_images_path + '/' + filename for filename in os.listdir(labeled_images_path)]

    return image_paths, label_paths

base_dataset_path = '/Users/gokaytoga/Downloads/Augmented_Data'
#/Downloads/Augmented_Data /Downloads/Augmented_Data
#/content/drive/MyDrive/Augmented_Data

# Create paths for training data
x_train, y_train = create_paths(base_dataset_path, 'train')
# Create paths for test data
x_test, y_test = create_paths(base_dataset_path, 'test')
# Create paths for val data
x_val, y_val = create_paths(base_dataset_path, 'val')

# Print the first few paths for verification
print(x_train[:5])
print(y_train[:5])


batch_size = 4
train_dataset = SegDataset3(x_train, y_train, batch_size=batch_size, transform = Aug_parameters)
test_dataset = SegDataset3(x_test, y_test, batch_size=batch_size, transform=None)  # No augmentation for the test set
val_dataset = SegDataset3(x_val, y_val, batch_size=batch_size, transform = Aug_parameters)

#THIS PART IS FOR USE WHEN COMPUTING PERFORMANCE PROBLEMS OCCUR
#target_size = (256, 256)
#train_dataset = SegDataset3(x_train, y_train, batch_size=batch_size, transform = Aug_parameters, target_size=target_size)
#test_dataset = SegDataset3(x_test, y_test, batch_size=batch_size, transform=None, target_size=target_size)  # No augmentation for the test set
#val_dataset = SegDataset3(x_val, y_val, batch_size=batch_size, transform = Aug_parameters, target_size=target_size)


image_datasets = {'train_set': train_dataset, 'test_set': test_dataset}
#/Downloads/Augmented_Data/train
#/Downloads/Augmented_Data/train
#/Downloads/Augmented_Data/train

['/Users/gokaytoga/Downloads/Augmented_Data/train/train-org-img/7078.jpg', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-org-img/6372.jpg', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-org-img/6414.jpg', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-org-img/7722.jpg', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-org-img/8411.jpg']
['/Users/gokaytoga/Downloads/Augmented_Data/train/train-label-img/7753_lab.png', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-label-img/6564_lab.png', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-label-img/6574_lab.png', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-label-img/8962_lab.png', '/Users/gokaytoga/Downloads/Augmented_Data/train/train-label-img/7270_lab.png']




*   The model:



In [ ]:
#### UNET CONFIG #########
ENCODER_DEPTH=5
DECODER_CHANNELS=(256, 128, 64, 32, 16)
BATCH_SIZE= [8, 16, 32, 64, 128]
LR = [1, 1e-2, 1e-4, 1e-6]
BATCH_SIZE = [8]
LR = [1e-2]
EPOCHS= 25

In [ ]:
def train_model(unique_name, num_epochs=EPOCHS, start_alpha_from=15, reach_max_alpha_in=655, max_alpha=0.5):
    for lr in LR:
        for bs in BATCH_SIZE:
            print("__" * 80)
            print("__" * 80)
            print(f"name: {unique_name} LR: {lr} BS: {bs}")
            print("__" * 80)
            print("__" * 80)


            # Create a model using TensorFlow/Keras
            Backbone = tf.keras.applications.ResNet101 #create_tf_model()  # Implement your TensorFlow model creation function

            # Compile the model with an optimizer and loss function
            optimizer = SGD(learning_rate=lr)
            model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

            # Create TensorBoard callback for logging
            tensorboard_callback = TensorBoard(log_dir=f'logs/{unique_name}-{lr}-{bs}', histogram_freq=1)

            # Define ModelCheckpoint callback for saving the best model
            model_checkpoint = ModelCheckpoint(filepath=f'models/{unique_name}-best-model.h5', save_best_only=True)

            # Training loop remains similar with TensorFlow-specific adjustments
            for epoch in range(num_epochs):
                # Rest of the training loop

                # Use model.fit() for training instead of manual optimization steps
                history = model.fit(train_dataset, epochs=1, callbacks=[tensorboard_callback, model_checkpoint])

                # Rest of the training loop

            # Save the final model
            model.save(f'models/{unique_name}-final-model.h5')

    return model

Tensorflow model:

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[713, 713, 3], include_top=False)

layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

def unet_model(output_channels:int):
  inputs = tf.keras.layers.Input(shape=[DIMENSIONS, DIMENSIONS, 3])

  # Downsampling through the model
  skips = down_stack(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

my Unet model:

In [ ]:
class ResizeLayer(layers.Layer):
    def __init__(self, target_shape, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_shape = target_shape

    def call(self, inputs):
        return tf.image.resize(inputs, self.target_shape)

    def get_config(self):
        config = super(ResizeLayer, self).get_config()
        config.update({'target_shape': self.target_shape})
        return config

def conv_block(x, filters, kernel_size, activation='relu'):
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    return x

def unet_model(input_shape=(713, 713, 3), num_classes=10):
    # Encoder
    inputs = tf.keras.Input(shape=input_shape)
    conv1 = conv_block(inputs, 64, 3)
    conv1 = conv_block(conv1, 64, 3)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128, 3)
    conv2 = conv_block(conv2, 128, 3)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256, 3)
    conv3 = conv_block(conv3, 256, 3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = conv_block(pool3, 512, 3)
    conv4 = conv_block(conv4, 512, 3)

    # Decoder
    up5 = layers.UpSampling2D(size=(2, 2))(conv4)
    resize5 = ResizeLayer(target_shape=(conv3.shape[1], conv3.shape[2]))(up5)
    concat5 = layers.Concatenate()([resize5, conv3])
    conv5 = conv_block(concat5, 256, 3)
    conv5 = conv_block(conv5, 256, 3)

    up6 = layers.UpSampling2D(size=(2, 2))(conv5)
    resize6 = ResizeLayer(target_shape=(conv2.shape[1], conv2.shape[2]))(up6)
    concat6 = layers.Concatenate()([resize6, conv2])
    conv6 = conv_block(concat6, 128, 3)
    conv6 = conv_block(conv6, 128, 3)

    up7 = layers.UpSampling2D(size=(2, 2))(conv6)
    resize7 = ResizeLayer(target_shape=(conv1.shape[1], conv1.shape[2]))(up7)
    concat7 = layers.Concatenate()([resize7, conv1])
    conv7 = conv_block(concat7, 64, 3)
    conv7 = conv_block(conv7, 64, 3)

    # Output layer
    output = layers.Conv2D(num_classes, 1, activation='softmax')(conv7)

    model = tf.keras.Model(inputs=inputs, outputs=output, name='unet_model')
    return model




*   Training the model:




In [ ]:
OUTPUT_CLASSES = 10

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['mIoU'])

In [ ]:
model = unet_model()

# Compile the model
model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'] )

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

# Train the model
#@tf.autograph.experimental.do_not_convert
model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, callbacks=callbacks)

In [ ]:
predictions = model.predict(test_dataset)



*   Visualization



In [ ]:
for image, true_mask in test_dataset.take(5):  # Visualize predictions for the first 5 samples
    predicted_mask = model.predict(image[tf.newaxis, ...])[0]

    # Visualize using matplotlib or other visualization libraries
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(image.numpy().astype('uint8'))
    plt.title('Input Image')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask.numpy().squeeze(), cmap='viridis', vmin=0, vmax=1)
    plt.title('True Mask')

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask.argmax(axis=-1), cmap='viridis', vmin=0, vmax=1)
    plt.title('Predicted Mask')

    plt.show()